In [ ]:
%pip install slack-sdk

In [ ]:
# Cell 1 — Imports y carga de estado

import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

SLACK_TOKEN   = os.environ["SLACK_BOT_TOKEN"]
SLACK_CHANNEL = os.environ.get("slack_channel", "#ocr-reports")
STATUS_PATH   = "status.json"

client = WebClient(token=SLACK_TOKEN)

# 1) Cargar status.json
with open(STATUS_FILE, 'r', encoding='utf-8') as f:
    statuses = json.load(f)

# 2) Calcular métricas
count_processed = sum(1 for e in statuses if e["status"] == "success")
count_errors= sum(1 for e in statuses if e["status"] == "error")
errors = [e for e in statuses if e["status"] == "error"]

In [ ]:
repo  = os.environ.get("lakefs_repository","vital-oceans")
commit = os.environ.get("lakefs_commit","92c647c3f1ac9bb79fdaae5860da56ad93b5eb4c522d5a328c30f7f7c700f7c3")
topic_id = os.environ.get("topic_id")

In [ ]:
# 4) Construir mensaje
text = (
    "*📊 Reporte final OCR*\n"
    f"• Repositorio: `{repo}`\n"
    f"• Commit:     `{commit}`\n"
    f"• Procesados: *{count_processed}* archivos\n"
    f"• Errores:    *{count_errors}* archivos\n"
    f"• Topic ID:     `{topic_id}`\n"
)

if count_errors > 0:
    text += "\n\n*Archivos con error:*\n"
    for err in errors[:10]:
        text += f"  – `{err['file']}`\n"
    if count_errors > 10:
        text += f"  _…y {count_errors - 10} más_\n"

# 5) Enviar a Slack
try:
    resp = client.chat_postMessage(
        channel=SLACK_CHANNEL,
        text=text
    )
    print(f"✅ Reporte enviado a {SLACK_CHANNEL} (ts={resp['ts']})")
except SlackApiError as e:
    print(f"❌ Error enviando a Slack: {e.response['error']}")